# Feed Forward Classification Network
To train this model, firs

In [ ]:
!pip install hyperas

In [ ]:
import h5py
from os.path import join,exists
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten
from tensorflow.keras.optimizers import RMSprop
from hyperas.distributions import choice
from hyperas import optim
from tensorflow.keras.constraints import max_norm
from random import randint
import numpy as np
from hyperopt import Trials, STATUS_OK, tpe
from sklearn.utils import shuffle
from tensorflow import keras
from sklearn.metrics import roc_curve,roc_auc_score
import os

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/synopsys-2022/models/classification')

In [ ]:
def create_model(X_train, Y_train, X_test, Y_test):
    W_maxnorm = 3
    DROPOUT = {{choice([0.3,0.5,0.7])}}
    # DROPOUT = 0.3

    model = Sequential()
    model.add(Flatten(input_shape=(20, 1, 20)))
    model.add(Dense(32,activation='relu',kernel_constraint=max_norm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(32,activation='relu',kernel_constraint=max_norm(W_maxnorm)))
    model.add(Dropout(DROPOUT))
    model.add(Dense(2,kernel_constraint=max_norm(W_maxnorm)))
    model.add(Activation('softmax'))

    myoptimizer = RMSprop(lr={{choice([1e-1,0.01,0.001,0.0001,1e-5])}}, rho=0.9, epsilon=1e-06)
    # myoptimizer = RMSprop(lr=1e-1, rho=0.9, epsilon=1e-06)
    mylossfunc = 'binary_crossentropy'
    model.compile(loss=mylossfunc, optimizer=myoptimizer,metrics=['accuracy'])
    # print(model.summary())
    # result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1, verbose=False)
    result = model.fit(X_train, Y_train, batch_size=100, epochs=5,validation_split=0.1,verbose=False)

    val_loss = np.amax(result.history['val_loss'])
    # score = model.evaluate(X_test,Y_test)
    # model_arch = 'MODEL_ARCH'
    # bestaccfile = join('TOPDIR',model_arch,model_arch+'_hyperbestacc')
    # reportAcc(acc,score,bestaccfile)

    return {'loss': val_loss, 'status': STATUS_OK, 'model': model}

In [ ]:
def train_data():
  data_train = h5py.File('./data/train.h5.batch1','r')
  X_train = np.array(data_train['data'])
  Y_train = np.array(data_train['label'])
  X_train_shuffled, Y_train_shuffled = shuffle(X_train, Y_train)

  data_test = h5py.File('./data/test.h5.batch1', 'r')
  X_test = np.array(data_test['data'])
  Y_test = np.array(data_test['label'])
  X_test_shuffled, Y_test_shuffled = shuffle(X_test, Y_test)
  return X_train_shuffled, Y_train_shuffled, X_test_shuffled, np.fliplr(Y_test_shuffled)

In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
X_train

array([[[[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 1., 0., 0.]],

        ...,

        [[0., 1., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]]],


       [[[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]]],


       ...,


       [[[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.]],



In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                          data=train_data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='seq_32_32',
                                          verbose=False)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)



100%|██████████| 10/10 [01:41<00:00, 10.17s/it, best loss: 0.14757832884788513]


In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
best_model.fit(X_train, Y_train, batch_size=100, epochs=20,validation_split=(0.1))

Epoch 1/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2779 - accuracy: 0.9135 - val_loss: 0.2559 - val_accuracy: 0.9205
Epoch 2/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2687 - accuracy: 0.9192 - val_loss: 0.2495 - val_accuracy: 0.9254
Epoch 3/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2644 - accuracy: 0.9218 - val_loss: 0.2442 - val_accuracy: 0.9289
Epoch 4/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2570 - accuracy: 0.9260 - val_loss: 0.2400 - val_accuracy: 0.9300
Epoch 5/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2518 - accuracy: 0.9285 - val_loss: 0.2364 - val_accuracy: 0.9308
Epoch 6/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2473 - accuracy: 0.9292 - val_loss: 0.2332 - val_accuracy: 0.9325
Epoch 7/20
571/571 [==============================] - 2s 4ms/step - loss: 0.2440 - accuracy: 0.9317 - val_loss: 0.2306 - val_accuracy: 0.9330
Epoch 

In [ ]:
y_pred = best_model.predict(X_test)
roc_auc_score(Y_test,y_pred)

0.964808161033258

In [ ]:
best_model.save('./seq_32_32')

INFO:tensorflow:Assets written to: ./seq_32_32/assets


In [ ]:
X_train, Y_train, X_test, Y_test = train_data()
loaded_model = keras.models.load_model('./seq_32_32')
y_pred = loaded_model.predict(X_test)

roc_auc_score(Y_test,y_pred)

0.9648081530644452

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
y_pred

array([[1.1593320e-03, 9.9884075e-01],
       [9.9999464e-01, 5.3178915e-06],
       [1.0756514e-02, 9.8924351e-01],
       ...,
       [9.9999905e-01, 9.5336418e-07],
       [1.2655136e-03, 9.9873453e-01],
       [8.2211645e-04, 9.9917787e-01]], dtype=float32)

In [ ]:
roc_auc_score(Y_test,y_pred)

0.9648081530644452

In [ ]:
(Y_test.argmax(axis=1) == y_pred.argmax(axis=1)).mean()

0.9677636298421808